In [3]:
import json
from bs4 import Tag
from tqdm import tqdm
import teducn_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [16]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

def scrape_article(url, page_key_code):
    try:
        article_content = teducn_utils.scrape_teducn_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_teducn_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"teducn_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            key_code_short = " ".join(page_key_list[1:-1])
            page_key_code = "Page " + key_code_short + " "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [12]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in teducn_ALL_link_སློབ་རའི་གསར་འགྱུར།.json: 1
page key name: Page སློབ་རའི་གསར་འགྱུར། 1
Error processing file teducn_ALL_link_སློབ་རའི་གསར་འགྱུར།.json: 'list' object has no attribute 'join'
Total page in teducn_ALL_link_སློབ་གསོ་གསར་འགྱུར།.json: 1
page key name: Page སློབ་གསོ་གསར་འགྱུར། 1
Error processing file teducn_ALL_link_སློབ་གསོ་གསར་འགྱུར།.json: 'list' object has no attribute 'join'
Total page in teducn_ALL_link_མི་རིགས་སྲིད་ཇུས།.json: 1
page key name: Page མི་རིགས་སྲིད་ཇུས། 1
Error processing file teducn_ALL_link_མི་རིགས་སྲིད་ཇུས།.json: 'list' object has no attribute 'join'
Total page in teducn_ALL_link_ས་གནས་བཅའ་ཁྲིམས།.json: 1
page key name: Page ས་གནས་བཅའ་ཁྲིམས། 1
Error processing file teducn_ALL_link_ས་གནས་བཅའ་ཁྲིམས།.json: 'list' object has no attribute 'join'
Total page in teducn_ALL_link_བསླབ་གཞིའི་སློབ་དེབ།.json: 1
page key name: Page བསླབ་གཞིའི་སློབ་དེབ། 1
Error processing file teducn_ALL_link_བསླབ་གཞིའི་སློབ་དེབ།.json: 'list' object has no attribute 'join'
To

100%|██████████| 47/47 [00:00<00:00, 38547.57it/s]

Error processing file teducn_ALL_link_ནང་བའི་གྲུབ་མཐའ།.json: 'list' object has no attribute 'join'
Error processing file teducn_ALL_link_ཁོར་ཡུག་དང་སྤྱི་ཚོགས།.json: 'list' object has no attribute 'join'
Error processing file teducn_ALL_link_ཁོར་ཡུག་དང་རིག་གནས།.json: 'list' object has no attribute 'join'
Error processing file teducn_ALL_link_རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།.json: 'list' object has no attribute 'join'
Total page in teducn_ALL_link_ཁོར་ཡུག་དང་སློབ་གསོ།.json: 1
page key name: Page ཁོར་ཡུག་དང་སློབ་གསོ། 1
Error processing file teducn_ALL_link_ཁོར་ཡུག་དང་སློབ་གསོ།.json: 'list' object has no attribute 'join'
Total page in teducn_ALL_link_མཁས་བའི་ཁྱུ་མཆོག.json: 10
page key name: Page མཁས་བའི་ཁྱུ་མཆོག 10
Error processing file teducn_ALL_link_མཁས་བའི་ཁྱུ་མཆོག.json: 'list' object has no attribute 'join'
Total page in teducn_ALL_link_དངོས་ལུགས།.json: 1
page key name: Page དངོས་ལུགས། 1
Total page in teducn_ALL_link_སློབ་འབྲིང་སློབ་གྲྭ།.json: 1
page key name: Page སློབ་འབྲིང་སློབ་གྲྭ། 1
To

In [ ]:
# 14:44

## fixing missing file

In [17]:
json_files = ["./data/links/teducn_ALL_link_རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།.json"]
    
with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))

print(f"Processed {len(json_files)} files")

Total page in teducn_ALL_link_རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།.json: 1
page key name: Page རོལ་དབྱངས། ལུས་རྩལ། རི་མོ། 1
Page key code: Page རོལ་དབྱངས། ལུས་རྩལ། རི་མོ། 


  6%|▌         | 1/18 [00:07<02:11,  7.75s/it]

<table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.com/index.php"> མདུན་ངོས།</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/">གཞུང་ལུགས་ཞིབ་འཇུག</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/yintimei/">རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།</a></span></td>
</tr>
</table><table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.com/index.php"> མདུན་ངོས།</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/">གཞུང་ལུགས་ཞིབ་འཇུག</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/yintimei/">རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།</a></span></td>
</tr>
</table>
['མདུན་ངོས།', 'གཞུང་ལུགས་ཞིབ་འཇུག', 'རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།']
<table border="0" cellpadding="0" cellspacing="0" class="title_info" width="100%">
<tr>
<td><span class="lan_19">༄༅། །བ


 28%|██▊       | 5/18 [00:08<00:16,  1.28s/it]

<table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.com/index.php"> མདུན་ངོས།</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/">གཞུང་ལུགས་ཞིབ་འཇུག</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/yintimei/">རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།</a></span></td>
</tr>
</table>
['མདུན་ངོས།', 'གཞུང་ལུགས་ཞིབ་འཇུག', 'རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།']
<table border="0" cellpadding="0" cellspacing="0" class="title_info" width="100%">
<tr>
<td><span class="lan_19">༄༅། །ཞང་བོད་ཀྱི་རི་སུལ་ནས་མཆེད་པའི་གནའ་བོའི་གླུ་གར་བྲོ་གསུམ་ལ་དཔྱད་པའི་གཏམ།</span></td>
</tr>
<tr>
<td class="hei_zw">སྤེལ་དུས།03-21  ཡོང་ཁུངས། ཧི་མ་ལ་ཡའི་བོན་དྲ་བ།  རྩོམ་པ་པོ། དགེ་བཤེས་བློ་གྲོས་རབ་གསལ།</td>
</tr>
</table>
<table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.co

...

<table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.com/index.php"> མདུན་ངོས།</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/">གཞུང་ལུགས་ཞིབ་འཇུག</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/yintimei/">རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།</a></span></td>
</tr>
</table>
['མདུན་ངོས།', 'གཞུང་ལུགས་ཞིབ་འཇུག', 'རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།']
<table border="0" cellpadding="0" cellspacing="0" class="title_info" width="100%">
<tr>
<td><span class="lan_19">༄༅། །བོད་ཀྱི་ཐག་འཐེན།</span></td>
</tr>
<tr>
<td class="hei_zw">སྤེལ་དུས།03-20  ཡོང་ཁུངས། བོད་ལྗོངས་ཞིབ་འཇུག  རྩོམ་པ་པོ།</td>
</tr>
</table>



 89%|████████▉ | 16/18 [02:30<00:13,  6.86s/it]

<table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.com/index.php"> མདུན་ངོས།</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/">གཞུང་ལུགས་ཞིབ་འཇུག</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/yintimei/">རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།</a></span></td>
</tr>
</table>
['མདུན་ངོས།', 'གཞུང་ལུགས་ཞིབ་འཇུག', 'རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།']
<table border="0" cellpadding="0" cellspacing="0" class="title_info" width="100%">
<tr>
<td><span class="lan_19">༄༅། །ཚག་དཀའ་མོ་རྡོ་འཕེན་པ།</span></td>
</tr>
<tr>
<td class="hei_zw">སྤེལ་དུས།03-20  ཡོང་ཁུངས།<a href="http://" target="_blank">http://www.teducn.com</a>  རྩོམ་པ་པོ། ས་གོང་དབང་འདུས། </td>
</tr>
</table>
<table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.com/index.php"> མདུན་ང


100%|██████████| 1/1 [02:33<00:00, 153.58s/it]

<table bgcolor="#DBF3FF" border="0" cellpadding="2" cellspacing="1" width="100%">
<tr>
<td bgcolor="#F0FBFF" height="30"><span class="lan_22">ད་སྔ་གཟིགས་བཞིན་པ། <a href="http://www.teducn.com/index.php"> མདུན་ངོས།</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/">གཞུང་ལུགས་ཞིབ་འཇུག</a> &gt; <a href="http://www.teducn.com/lilunyanjiu/yintimei/">རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།</a></span></td>
</tr>
</table>
['མདུན་ངོས།', 'གཞུང་ལུགས་ཞིབ་འཇུག', 'རོལ་དབྱངས། ལུས་རྩལ། རི་མོ།']
<table border="0" cellpadding="0" cellspacing="0" class="title_info" width="100%">
<tr>
<td><span class="lan_19">༄༅། །རྟ་ཐོག་མདའ་རྩལ་།</span></td>
</tr>
<tr>
<td class="hei_zw">སྤེལ་དུས།03-20  ཡོང་ཁུངས།<a href="http://" target="_blank">http://www.teducn.com</a>  རྩོམ་པ་པོ། ས་གོང་དབང་འདུས། </td>
</tr>
</table>
Total Failure in the རོལ་དབྱངས། article: 0
teducn_ALL_content_རོལ་དབྱངས།.json
Successfully saved: teducn_ALL_content_རོལ་དབྱངས།.json
Processed 1 files


In [ ]:
len(json_files)